In [ ]:
import sys
import pandas as pd
sys.path.append('../utils/')
from utils import get_lib_gdf, get_ct_gdf, assignment_dataframes1, assignment_dataframes2
from plotting import plotting

In [ ]:
# mapbox constants
MAPBOX_TOKEN = ""

#### Create library and census tract geodataframe

In [ ]:
lib_file_path = "../data/library_locations.csv"
libs = get_lib_gdf(lib_file_path)

In [ ]:
ct_file_path = "../data/census_tract_boundaries.geojson"
cts = get_ct_gdf(ct_file_path)

#### Assign census tracts overlapping with lib 1 mile bufferzone

In [ ]:
mile1 = assignment_dataframes2(cts, libs, meters=1609)

#### Assign census tracts outside original buffer zone
For those census tracts that did not overlap with a bufferzone, expand the buffer and assign them

In [ ]:
remaining = pd.merge(cts, mile1.loc[:, ['namelsad10', 'name']], on='namelsad10', how='left')
remaining = remaining[remaining.isnull().any(axis=1)].reset_index()
remaining = remaining[~remaining.loc[:, 'namelsad10'].isin(['Census Tract 9800','Census Tract 7706.02'])]
remaining = remaining.drop(columns=['index', 'name', 'centroid'])

In [ ]:
mile5 = assignment_dataframes2(remaining, libs, meters=9654) #9654

#### Merge datasets for data visualizations

In [ ]:
joined_df = mile1.append(mile5, ignore_index=True)

In [ ]:
plotting(joined_df, cts)